<a href="https://colab.research.google.com/github/proap900/AI_LAB/blob/main/ann_single_perceptron_and_back_prpagation_ai_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Single perceptron
import numpy as np
class Perceptron:
    def __init__(self, learning_rate=0.1, epochs=100):
        self.lr = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None


    def activation(self, z):
        return 1 if z >= 0 \
            else 0  # Step function

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.epochs):
            for idx, x_i in enumerate(X):
                linear_output = np.dot(x_i, self.weights) + self.bias
                y_pred = self.activation(linear_output)
                error = y[idx] - y_pred
                self.weights += self.lr * error * x_i
                self.bias += self.lr * error

    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        return [self.activation(z) for z in linear_output]

# Define the logic gates
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_and = np.array([0, 0, 0, 1])  # AND
y_or = np.array([0, 1, 1, 1])   # OR
y_xor = np.array([0, 1, 1, 0])   # XOR (not linearly separable)

# Train and test AND gate
print("\n--- AND Gate ---")
perceptron_and = Perceptron()
perceptron_and.fit(X, y_and)
print("Weights:", perceptron_and.weights, "Bias:", perceptron_and.bias)
print("Predictions:", perceptron_and.predict(X))

# Train and test OR gate
print("\n--- OR Gate ---")
perceptron_or = Perceptron()
perceptron_or.fit(X, y_or)
print("Weights:", perceptron_or.weights, "Bias:", perceptron_or.bias)
print("Predictions:", perceptron_or.predict(X))

# Try XOR (will fail)
print("\n--- XOR Gate (Will Fail) ---")
perceptron_xor = Perceptron(epochs=1000)  # Even with more epochs, it fails
perceptron_xor.fit(X, y_xor)
print("Weights:", perceptron_xor.weights, "Bias:", perceptron_xor.bias)
print("Predictions:", perceptron_xor.predict(X))


--- AND Gate ---
Weights: [0.2 0.1] Bias: -0.20000000000000004
Predictions: [0, 0, 0, 1]

--- OR Gate ---
Weights: [0.1 0.1] Bias: -0.1
Predictions: [0, 1, 1, 1]

--- XOR Gate (Will Fail) ---
Weights: [-0.1  0. ] Bias: 0.0
Predictions: [1, 1, 0, 0]


In [2]:
# Back Propagation
import numpy as np
class NeuralNetwork:
    def __init__(self):
        # Network architecture (2-2-1)
        self.input_size = 2
        self.hidden_size = 2
        self.output_size = 1

        # Initialize weights with random values
        self.W1 = np.random.randn(self.input_size, self.hidden_size)  # Input to hidden
        self.W2 = np.random.randn(self.hidden_size, self.output_size)  # Hidden to output
        self.b1 = np.zeros((1, self.hidden_size))  # Hidden layer bias
        self.b2 = np.zeros((1, self.output_size))  # Output layer bias

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def forward(self, X):
        # Forward propagation
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)  # Hidden layer activation
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.sigmoid(self.z2)  # Output layer activation
        return self.a2

    def backward(self, X, y, output, learning_rate=0.1):
        # Backward propagation
        self.error = y - output
        self.delta2 = self.error * self.sigmoid_derivative(output)
        self.error_hidden = self.delta2.dot(self.W2.T)
        self.delta1 = self.error_hidden * self.sigmoid_derivative(self.a1)

        # Update weights and biases
        self.W2 += self.a1.T.dot(self.delta2) * learning_rate
        self.b2 += np.sum(self.delta2, axis=0, keepdims=True) * learning_rate
        self.W1 += X.T.dot(self.delta1) * learning_rate
        self.b1 += np.sum(self.delta1, axis=0) * learning_rate

    def train(self, X, y, epochs=10000, learning_rate=0.1):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(X, y, output, learning_rate)

            if epoch % 1000 == 0:
                loss = np.mean(np.square(y - output))
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        return np.round(self.forward(X))


# XOR dataset
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Create and train the neural network
nn = NeuralNetwork()
print("Initial predictions (before training):")
print(nn.predict(X))

nn.train(X, y, epochs=10000, learning_rate=0.1)

print("\nFinal predictions (after training):")
print(nn.predict(X))
print("\nFinal weights and biases:")
print("W1:", nn.W1)
print("b1:", nn.b1)
print("W2:", nn.W2)
print("b2:", nn.b2)

Initial predictions (before training):
[[1.]
 [0.]
 [1.]
 [1.]]
Epoch 0, Loss: 0.2508
Epoch 1000, Loss: 0.2490
Epoch 2000, Loss: 0.2329
Epoch 3000, Loss: 0.1710
Epoch 4000, Loss: 0.0680
Epoch 5000, Loss: 0.0207
Epoch 6000, Loss: 0.0103
Epoch 7000, Loss: 0.0065
Epoch 8000, Loss: 0.0047
Epoch 9000, Loss: 0.0036

Final predictions (after training):
[[0.]
 [1.]
 [1.]
 [0.]]

Final weights and biases:
W1: [[-5.60806705 -3.98474447]
 [-5.60006916 -3.98314215]]
b1: [[2.11713022 5.88497186]]
W2: [[-8.01492466]
 [ 7.66143903]]
b2: [[-3.51133878]]
